In [1]:
import warnings
warnings.simplefilter("ignore")

In [2]:
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model

import opacus
from opacus.validators import ModuleValidator
from opacus.utils.batch_memory_manager import BatchMemoryManager
from opacus import PrivacyEngine

import torch
import torch.nn as nn
import numpy as np

from tqdm.notebook import tqdm
from torch.optim import SGD
from torch.utils.data import DataLoader

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig

from sklearn.metrics import accuracy_score

2023-11-15 09:09:02.212346: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 09:09:02.501720: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-15 09:09:02.507985: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:/opt/software/Java/1.8.0_152/lib:/opt/software/Python/3.

In [3]:
model_name = "prajjwal1/bert-tiny"
EPOCHS = 10
BATCH_SIZE = 2048
LR = 0.005

In [4]:
# Prepare data
dataset = load_dataset("glue", "qqp")
num_labels = dataset["train"].features["label"].num_classes

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
tokenized_dataset = dataset.map(
    lambda example: tokenizer(
        example["question1"],
        example["question2"],
        max_length=128,
        padding='max_length',
        truncation=True
    ),
    batched=True
)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [7]:
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [8]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 363846
    })
    validation: Dataset({
        features: ['question1', 'question2', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40430
    })
    test: Dataset({
        features: ['question1', 'question2', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 390965
    })
})

In [9]:
train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=BATCH_SIZE)

In [10]:
EPSILON = 8.0
DELTA = 1/len(train_dataloader)
MAX_GRAD_NORM = 0.5
MAX_PHYSICAL_BATCH_SIZE = int(BATCH_SIZE/4)

In [11]:
config = AutoConfig.from_pretrained(model_name)
config.num_labels = num_labels

model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, 
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules = ['query', 'key', 'value'],
)

if peft_config is not None:
    model = get_peft_model(model, peft_config)
    model.register_full_backward_hook(True)

device = torch.device("cuda:0")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 128, padding_idx=0)
          (position_embeddings): Embedding(512, 128)
          (token_type_embeddings): Embedding(2, 128)
          (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-1): 2 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(
                    in_features=128, out_features=128, bias=True
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=128, out_fe

In [12]:
errors = ModuleValidator.validate(model, strict=False)
print(errors)

[]


In [13]:
optimizer = SGD(params=model.parameters(), lr=LR)

In [14]:
privacy_engine = PrivacyEngine(accountant="rdp")

model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
    batch_first=True,
)

In [15]:
print(f"Using Sigma = {optimizer.noise_multiplier:.3f} | C = {optimizer.max_grad_norm} | Initial DP (ε, δ) = ({privacy_engine.get_epsilon(DELTA)}, {DELTA})")

Using Sigma = 0.479 | C = 0.5 | Initial DP (ε, δ) = (0, 0.0056179775280898875)


In [16]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable Parameters: {trainable_params} || All Parameters: {all_param} || Trainable Parameters (%): {100 * trainable_params / all_param:.2f}"
    )

print_trainable_parameters(model)

Trainable Parameters: 12546 || All Parameters: 4398724 || Trainable Parameters (%): 0.29


In [17]:
def train(model, train_dataloader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    epsilon = []

    with BatchMemoryManager(
        data_loader=train_dataloader, 
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE,
        optimizer=optimizer,
        ) as memory_safe_data_loader:

        for i, batch in tqdm(enumerate(memory_safe_data_loader), total=len(memory_safe_data_loader), desc=f"Training Epoch: {epoch}"):
            
            batch = {k: v.to(device) for k, v in batch.items()}
            optimizer.zero_grad()

            outputs = model(**batch)
            loss = criterion(outputs.logits, batch["labels"])
            loss.backward()

            optimizer.step()
            losses.append(loss.item())

            if i % 8000 == 0:
                epsilon = privacy_engine.get_epsilon(DELTA)

                print(f"Training Epoch: {epoch} | Loss: {np.mean(losses):.6f} | ε = {epsilon:.2f}")                    

In [18]:
def test(model, test_dataloader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()

    losses = []
    accuracies = []

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Test"):
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
            loss = criterion(outputs.logits, batch["labels"])

            preds = outputs.logits.argmax(dim=-1)
            acc = accuracy_score(preds.cpu().numpy(), batch["labels"].cpu().numpy())

            losses.append(loss.item())
            accuracies.append(acc.item())

    acc = np.mean(accuracies)
    loss = np.mean(losses)

    print(
        f"Test set: Loss: {loss:.4f}, Accuracy: {acc*100:.2f}%"
    )

    return loss, acc

In [19]:
for epoch in tqdm(range(EPOCHS), desc=f'Training {EPOCHS} Epochs'):
    train(model, train_dataloader, optimizer, epoch + 1, device)

Training 10 Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Training Epoch: 1:   0%|          | 0/710 [00:00<?, ?it/s]

Training Epoch: 1 | Loss: 0.699795 | ε = 0.00


Training Epoch: 2:   0%|          | 0/710 [00:00<?, ?it/s]

Training Epoch: 2 | Loss: 0.669002 | ε = 3.48


...

Training Epoch: 8 | Loss: 1.497815 | ε = 6.82


Training Epoch: 9:   0%|          | 0/710 [00:00<?, ?it/s]

Training Epoch: 9 | Loss: 1.427647 | ε = 7.25


Training Epoch: 10:   0%|          | 0/710 [00:00<?, ?it/s]

Training Epoch: 10 | Loss: 1.495963 | ε = 7.62


In [20]:
final_epsilon = privacy_engine.get_epsilon(DELTA)
print(f"Final DP Guarantee (ε, δ)-DP = ({final_epsilon:.2f}, {DELTA})")

Final DP Guarantee (ε, δ)-DP = (7.99, 0.0056179775280898875)


In [21]:
test(model, test_dataloader, device)

Test:   0%|          | 0/20 [00:00<?, ?it/s]

Test set: Loss: 1.4513, Accuracy: 63.19%


(1.4513079524040222, 0.6319340440752634)